In [92]:
import os
import re
earlygreekepic = ['tlg0012','tlg0020','tlg0013']

def getepicfiles():
    rlist = []
    for (root,dirs,files) in os.walk('.'):
        m = re.search('(tlg[0-9]+)',root)
        if(not m):
            continue
        if(m and not m[1] in earlygreekepic):
            continue
        for file in files:
            if(re.search('tlg0012.tlg00[12].daphne_tb',file)):
                continue
            if(re.search('tlg0020.tlg00[123].daphne_tb',file)):
                continue
            if(re.search('\.bak',file)):
                continue
            if(re.search('conllu$',file)):
                #print(root,file)
                rlist.append(root+'/'+file)
    return(rlist)

flist = getepicfiles()
flist

['./tlg0020/tlg0020.tlg002.daphne_perstb-grc1.conllu',
 './tlg0020/tlg0020.tlg003.daphne_perstb-grc1.conllu',
 './tlg0020/tlg0020.tlg001.daphne_perstb-grc1.conllu',
 './tlg0012/tlg001/tlg0012.tlg001.daphne_perstb-grc1.1-6.conllu',
 './tlg0012/tlg001/tlg0012.tlg001.daphne_perstb-grc1.7-12.conllu',
 './tlg0012/tlg001/tlg0012.tlg001.daphne_perstb-grc1.13-18.conllu',
 './tlg0012/tlg001/tlg0012.tlg001.daphne_perstb-grc1.19-24.conllu',
 './tlg0012/tlg002/tlg0012.tlg002.daphne_perstb-grc1.7-12.conllu',
 './tlg0012/tlg002/tlg0012.tlg002.daphne_perstb-grc1.19-24.conllu',
 './tlg0012/tlg002/tlg0012.tlg002.daphne_perstb-grc1.1-6.conllu',
 './tlg0012/tlg002/tlg0012.tlg002.daphne_perstb-grc1.13-18.conllu',
 './tlg0013/tlg0013.tlg002.daphne_tb-grc1.conllu']

In [95]:
wordid2cit = {}
wordid2form= {}
wordid2lemma = {}
wordid2upos = {}
wordid2xpos = {}
wordid2feats = {}
wordid2head = {}
wordid2deprel = {}
wordid2deplist = {}
lemmaindex = {}


def     add2lemmaindex(lemmaindex,lemma,wordid):
    if(not lemma in lemmaindex):
        lemmaindex[lemma] = [wordid]
    else:
        lemmaindex[lemma].append(wordid)

def loadUDfile(fname):
    cursentid = ''
    m = re.search('(tlg[0-9]+\.tlg[0-9]+)',fname)
    if(not m):
        print(fname,'expected file name with tlg[0-9]+\.tlg[0-9]+')
        return

    else:
        curtgwork = m[1]
    f = open(fname)
    for l in f:
        l = re.sub('\s+$','',l)
        if(re.search('u--------',l)):
            continue
        curcit = ''
        m = re.search('sent_id\s*=\s*([0-9a-b]+)',l)
        if(m):
            cursentid = m[1]
        m = re.search('^([0-9]+)',l)
        if(not m):
            continue
            
        wordid = curtgwork + ':sent' + cursentid + ':' + m[1]
        m = re.search('Ref=([^\|]+)',l)
        if(m):
            curcit = curtgwork + ':' + m[1]
        wordid2cit[wordid] = curcit
        args = l.split()
        #print(len(args),l)
        wordid2form[wordid] = args[1]
        wordid2lemma[wordid] = args[2]
        add2lemmaindex(lemmaindex,args[2],wordid)
        wordid2upos[wordid] = args[3]
        wordid2xpos[wordid] = args[4]
        wordid2feats[wordid] = args[5]
        wordid2head[wordid] = curtgwork + ':sent' + cursentid + ':' + args[6]
        if(not wordid2head[wordid] in wordid2deplist):
            wordid2deplist[wordid2head[wordid]] = [wordid]
        else:
            wordid2deplist[wordid2head[wordid]].append(wordid)
        wordid2deprel[wordid] = args[7]
        args = l.split()

        #print(wordid,wordid2form[wordid],wordid2head[wordid])
        
        
    f.close()

lemmadeps = {}
def getdeps(lemma):
    if(not lemma in lemmaindex):
        return

    print(lemma,len(lemmaindex[lemma]))
    for wordid in lemmaindex[lemma]:
        if(not wordid in wordid2deplist):
            continue
        print('\n'+wordid,wordid2form[wordid],wordid2lemma[wordid])
        for depid in wordid2deplist[wordid]:
            print(depid,wordid2lemma[depid],wordid2form[depid],wordid2deprel[depid])
            
    
loadUDfile(flist[0])
getdeps('ἔχω')
for wordid in wordid2head:
    #print(wordid)
    if(re.search(':0$',wordid2head[wordid])):
        continue
    #print(wordid,wordid2form[wordid],wordid2form[wordid2head[wordid]])

ἔχω 24

tlg0020.tlg002:sent1715898:5 ἔχουσιν ἔχω
tlg0020.tlg002:sent1715898:2 δέ δ̓ cc
tlg0020.tlg002:sent1715898:3 ἄνδιχα ἄνδιχα advmod
tlg0020.tlg002:sent1715898:4 θυμός θυμὸν obj

tlg0020.tlg002:sent1715914:3 ἔχουσι ἔχω
tlg0020.tlg002:sent1715914:1 κρύπτω κρύψαντες advcl
tlg0020.tlg002:sent1715914:2 γάρ γὰρ advmod
tlg0020.tlg002:sent1715914:4 θεός θεοὶ nsubj
tlg0020.tlg002:sent1715914:5 βίος βίον obj
tlg0020.tlg002:sent1715914:6 ἄνθρωπος ἀνθρώποισιν obl

tlg0020.tlg002:sent1715915:13 ἔχειν ἔχω
tlg0020.tlg002:sent1715915:9 ὥστε ὥστε mark
tlg0020.tlg002:sent1715915:10 σύ σε nsubj
tlg0020.tlg002:sent1715915:12 ἐνιαυτός ἐνιαυτὸν obl
tlg0020.tlg002:sent1715915:15 ἀεργός ἀεργὸν advcl

tlg0020.tlg002:sent1715938:19 ἔχοντες ἔχω
tlg0020.tlg002:sent1715938:16 πᾶς πάντες det
tlg0020.tlg002:sent1715938:18 δῶμα δώματ̓ obj

tlg0020.tlg002:sent1715941:8 εἶχ' ἔχω
tlg0020.tlg002:sent1715941:5 ὅτε ὅτε advmod
tlg0020.tlg002:sent1715941:6 δή δὴ advmod
tlg0020.tlg002:sent1715941:7 κακός κακὸν obj

tlg00

In [91]:
lemmaindex['μάχομαι']

KeyError: 'μάχομαι'

In [79]:
lemmaindex

{}